In [1]:
%load_ext autoreload
%autoreload 2

from fastai.text.all import *
from fastai.vision.all import *
import pandas as pd
import torch
from tqdm.notebook import tqdm

from utils import get_dls

In [2]:
seed = 42

# python RNG
import random
random.seed(seed)

# pytorch RNGs
import torch
torch.manual_seed(seed)
torch.backends.cudnn.deterministic = True
if torch.cuda.is_available(): torch.cuda.manual_seed_all(seed)

# numpy RNG
import numpy as np
np.random.seed(seed)

# tensorflow RNG
# tf.random.set_seed(seed)

In [3]:
data_path = Path("/mnt/nas/backups/08-07-2020/desktopg01/lisa/Data/CSV")

In [4]:
train = pd.read_csv(data_path/"train_small.csv")
val = pd.read_csv(data_path/"validation_small.csv")
test_data = pd.read_csv(data_path/"test_small.csv")

In [34]:
total = pd.concat([train, val, test_data])

In [35]:
total["length"] = total['body'].str.split().str.len()

In [36]:
total.head()

,themes,process_id,file_name,document_type,pages,body,length
0,[232],AI_856934,AI_856934_1926210_1060_17072013.pdf,outros,1,"{""tribunal justiça estado bahia poder judiciário salvador juizado cível defesa consumidor universo matutino projudi acm campus universidade salgado oliveira universo pituba salvador processo parte autora nailton lantyer cordeiro araujo parte atlantico fundo investimento direitos creditorios despacho vistos etc indefiro requerido pela parte demandante aguarde a sessão conciliação designada salvador de julho paulo alberto nunes chenaud juiz direito documento assinado eletronicamente""}",60
1,[232],AI_856934,AI_856934_1926211_34_17072013.pdf,outros,1,"{""excelentíssimo senhor doutor juiz direito juizado cível defesa consumidor universo comarca salvador autos processo º atlântico fundo investimento direitos creditórios não padronizados nova denominação crdg fundo investimento direitos creditórios não padronizados empresa inscrita cnpj sob º com sede av brigadeiro faria lima andar são paulo por seus advogados signatários nos autos ação epígrafe proposta por nailton lantyer cordeiro araujo vem respeitosamente presença vossa excelência inconformado com r sentença fls com fulcro ARTIGO_41 lei º interpor presente recurso inominado ser recebido...",183
2,[232],AI_856934,AI_856934_1926211_34_17072013.pdf,outros,2,"{""razões recurso inominado recorrente atlantico fundo investimento direitos creditórios não padronizados recorrido nailton lantyer cordeiro araujo autos processo º origem jec comarca salvador colendo colégio recursal ilustres julgadores síntese demanda parte recorrida intentou presente demanda alegando síntese que teve seu nome indevidamente incluído cadastro proteção crédito que diligenciar sentido obter informações acerca inscrição verificou que negativação foi ordenada pelo recorrente sendo assim propôs presente demanda pleiteando exclusão seus dados spc serasa inexigibilidade débito o ...",154
3,[232],AI_856934,AI_856934_1926211_34_17072013.pdf,outros,3,"{""empresa recorrente tornou credora dos débitos nome parte recorrida referente uma linha telefônica razão contrato cessão crédito entre empresa originalmente credora cedente o atlântico fundo investimentos direitos creditórios não padronizados ora cessionária com efeito caso não houve qualquer cobrança indevida afinal tornando a parte recorrida inadimplente vindo parte recorrente firmar contrato cessão direitos creditórios junto empresa credora recorrido totalmente legítima cobrança perpetrada cumpre esclarecer que época cessão parte recorrida estava inadimplente com empresa recorrente mot...",235
4,[232],AI_856934,AI_856934_1926211_34_17072013.pdf,outros,4,"{""entretanto verdade parte apelante tornou titular crédito devido pela parte apelada por força um contrato cessão crédito celebrado com brasil telecom a seja que pese parte apelada não ter firmado inicialmente contrato diretamente junto apelante este tornou seu credor virtude aludido instrumento cessão crédito portanto caso seja comprovada alegação perpetrada pela parte apelada que o contrato que originou seu débito seja fraudulento quem deverá arcar com eventuais ônus a brasil telecom a não esta parte apelante que não foi quem contratou com parte apelada por sua vez parte apelada foi regu...",256


In [38]:
total["length"].describe()

count    339478.000000
mean        174.560016
std         106.194949
min           1.000000
25%          92.000000
50%         171.000000
75%         240.000000
max        3304.000000
Name: length, dtype: float64

In [48]:
(total["length"] <= 500).mean()

0.9943648778418631

In [5]:
train_counts = train["document_type"].value_counts()
val_counts = val["document_type"].value_counts()
test_counts = test_data["document_type"].value_counts()
train_counts, val_counts, test_counts 

(outros                              134134
 peticao_do_RE                         9509
 agravo_em_recurso_extraordinario      2546
 sentenca                              2129
 acordao_de_2_instancia                 553
 despacho_de_admissibilidade            346
 Name: document_type, dtype: int64,
 outros                              84104
 peticao_do_RE                        6364
 agravo_em_recurso_extraordinario     2149
 sentenca                             1636
 acordao_de_2_instancia                299
 despacho_de_admissibilidade           183
 Name: document_type, dtype: int64,
 outros                              85408
 peticao_do_RE                        6331
 agravo_em_recurso_extraordinario     1841
 sentenca                             1475
 acordao_de_2_instancia                273
 despacho_de_admissibilidade           198
 Name: document_type, dtype: int64)

In [12]:
train_no_dupls = train.drop_duplicates()
val_no_dupls = val.drop_duplicates()
test_no_dupls = test_data.drop_duplicates()

In [13]:
train_counts = train_no_dupls["document_type"].value_counts()
val_counts = val_no_dupls["document_type"].value_counts()
test_counts = test_no_dupls["document_type"].value_counts()
train_counts, val_counts, test_counts 

(outros                              130337
 peticao_do_RE                         9465
 agravo_em_recurso_extraordinario      2508
 sentenca                              2129
 acordao_de_2_instancia                 553
 despacho_de_admissibilidade            346
 Name: document_type, dtype: int64,
 outros                              79936
 peticao_do_RE                        6180
 agravo_em_recurso_extraordinario     2001
 sentenca                             1568
 acordao_de_2_instancia                293
 despacho_de_admissibilidade           177
 Name: document_type, dtype: int64,
 outros                              80781
 peticao_do_RE                        6122
 agravo_em_recurso_extraordinario     1723
 sentenca                             1450
 acordao_de_2_instancia                271
 despacho_de_admissibilidade           197
 Name: document_type, dtype: int64)

In [3]:
path = Path("/mnt/nas/backups/08-07-2020/desktopg01/lisa/Data/small_flow")
dls = get_dls(path, 64, 224)

In [4]:
test_items = get_image_files(path, folders="test")

In [5]:
test_dl = dls.test_dl(test_items, with_labels=True)

In [21]:
dls.categorize.vocab

['acordao_de_2_instancia', 'agravo_em_recurso_extraordinario', 'despacho_de_admissibilidade', 'outros', 'peticao_do_RE', 'sentenca']

In [16]:
torch.bincount(tensor(dls.train.tls[1]))

tensor([   583,   4220,    361, 140786,  10181,   2177])

In [17]:
torch.bincount(tensor(dls.valid.tls[1]))

tensor([  314,  2650,   183, 91434,  6803,  1613])

In [18]:
torch.bincount(tensor(test_dl.tls[1]))

tensor([  285,  2537,   198, 87902,  6177,  1478])